## Import packages

In [ ]:
# First import matplotlib (for plotting) and skan
from matplotlib import pyplot as plt
%matplotlib inline
from skan import draw
import numpy as np

# Then import the radial hydride packagess
from RHaP import import_image
from RHaP import cropping_functions as crop
from RHaP import plot_functions as plt_f
from RHaP import radial_hydride_fraction as RHF
from RHaP import branching as branch
from RHaP import crack_path as cp
from RHaP import image_processing

## Importing Image

- First, import the image using the `import_image` command. Transpose the image if necessary using the `transpose` argument to make the radial direction vertical. 
- The `cropImage` function applies a rectangular crop to the image to remove scale bars, or if you have a specific rectangular region you want to look at. 
- Define the scale of the image (in pixels per meter) as `scale` in order to overlay scale bar.

In [ ]:
# Load image
original_image = import_image.image(image_path ='data/example.bmp', transpose = True)
cropped_image = crop.cropImage(original_image, crop_bottom=240, crop_top=0, crop_left=0, crop_right=0)

# Set scale: 0.1 mm = 162 pixels
scale = 0.1e-3 / 162
location = 'lower right'

# Plot image
plt_f.plot(img=cropped_image, title='Loaded image', scale=scale, location=location)

## Additional Cropping

The second crop function is `cropping_tube`, which should be used if the micrograph is curved and removes black pats of the image which are not the tube.

In [ ]:
# Crop tube
cropped_image, crop_threshold = crop.cropping_tube(cropped_image, 
                                                   crop_param = 0.1, size_param = 500, dilation_param = 5)

# Plot comparison
plt_f.plot_comparison(original_image, 'Original image crop', cropped_image, 'Tube crop', 
                      scale=scale, location=location)

## Image Processing

Grain contast or uneven lighting can be minimised through the application of a gaussian blur in the `minimize_grain_contrast` function.

In [ ]:
# Remove grain contrast
removed_grains = image_processing.minimize_grain_contrast(cropped_image, sigma=10)

# Plot image
plt_f.plot(img=cropped_image, title='Minimised grain contrast', scale=scale, location=location)

# Plot the histogram for removed grains so that we can see where we should threshold 
histogram = plt_f.plot_hist(removed_grains)

# Print an approximate threshold value which should work well
print('Approximate threshold: {0:.3f}'.format(
    2*np.nanmedian(removed_grains)-np.nanpercentile(removed_grains, 98)))

## Thresholding

After this, the image is thresholded using the `simple_threshold` function. The threshold value should be set using the `threshold` argument. Small features, less than a given size `small_obj` can optionally be removed. Note it is important not too over threshold the image, guidance of a value to threshold cis shown above and can be determined by investigating the histograms plotted above.

In [ ]:
# Apply threshold
thres = image_processing.simple_threshold(removed_grains, crop_threshold, 
                                          threshold = 0.820, small_obj = 50)

# Plot the thresholded image and compare it to the original image: 
plt_f.plot_comparison(cropped_image, 'Original Image', thres,'Thresholded Image', 
                      scale=scale, location=location)

## Radial Hydride Fraction Calculation

The first step is to perform the hough line transform `hough_rad` there are a few input parameters that should be considered: 
- `num_peaks`: should be changed dependent on the type of micrograph, if your hydrides are straight and not very interconnected a small value of around 2 is good, if in one box, there are many branches that need to be picked up, this value should be increased accordingly.
- `min_dist`, `min_angle` and `val` are pre-set and seem to work for most cases.
        

In [ ]:
# Apply Hough transform
angle_list,len_list = RHF.hough_rad(thres, num_peaks=2, scale=scale, location=location)

In [ ]:
#Non weighted radial hydride fraction
radial, circumferential = RHF.RHF_no_weighting_factor(angle_list, len_list)

print('The non-weighted RHF is {0:.4f}'.format(radial))

In [ ]:
#Weighted Radial Hydride Fraction 
fraction = RHF.weighted_RHF_calculation(angle_list, len_list)

print('The weighted RHF is: {0:.4f}'.format(fraction))

## Branch Length Fraction

Here we want to determine the extent of branching within the microstrucutre, this is done in two ways: 
- In image form where the branches are coloured differently to the main hydride
- BLF the length fraction of branches with respect to the toatal length of all hydrides in the microstrucutre

In [ ]:
# Calculate the branch length fraction
skel,is_main,BLF = branch.branch_classification(thres);

# Plot branching image
fig, ax = plt.subplots(figsize=(10,6))
ax = draw.overlay_skeleton_2d_class(
    skel,
    skeleton_color_source=lambda s: is_main,
    skeleton_colormap='spring',
    axes=ax
     )

plt.axis('off')
plt.title('Branched hydrides')
plt_f.addScaleBar(ax[0], scale=scale, location=location)
plt_f.addArrows(ax[0])

print('The BLF is: {0:.4f}'.format(BLF))

## Crack Path

Here we want to determine potential crack paths through the microstrucutre, we input the thresholded image `thres`. After running once, the area around that path (radius set with `kernel_size`) is discounted, then the process is repeated `num_runs` times.

In [ ]:
# Determing potential crack paths
edist, path_list, cost_list = cp.det_crack_path(thres, crop_threshold, num_runs=10, kernel_size=20)

In [ ]:
# Plot possible crack paths
fig, ax = plt.subplots(figsize=(10,6))
list_costs = []

for n, (p, c) in enumerate(zip(path_list, cost_list)):
    
    im = ax.imshow(thres, cmap='gray')
    
    if n==0:
        plt.colorbar(im,fraction=0.03, pad=0.01)
    ax.scatter(p[:,1], p[:,0], s=2, alpha=0.1)
    ax.text(p[-1][1], p[-1][0], s=str(n), c='w', bbox=dict(facecolor='black', edgecolor='black'))
    plt.axis('off')
    print('Run #{0}\tCost = {1:.2f}'.format(n,c))
    list_costs.append(c)
    
plt_f.addScaleBar(ax, scale=scale, location=location)
plt_f.addArrows(ax)

In [ ]:
# Histograms for plotting the costs of each path
plt.hist(list_costs, bins=5, cumulative = True, color = "cornflowerblue", ec="cornflowerblue", label = "Cumulative Distribution Function")
plt.hist(list_costs, bins=5, color = "lightpink", ec="lightpink", label = "Normal Distribution Function")
plt.legend()
plt.xlabel('Cost', fontsize="12")
plt.ylabel('Frequency',fontsize="12")
plt.title('Paths of Lowest Cost', fontweight="bold", fontsize="15")
plt.show()